In [5]:
from pyspark import SparkContext
#sc = SparkContext()
print(sc)
print(sc.version)

<SparkContext master=local[*] appName=pyspark-shell>
2.4.4


In [6]:
from pyspark.sql import SparkSession
my_spark = SparkSession.builder.getOrCreate()
print(my_spark)

In [7]:
my_spark.catalog.listTables()

[]

In [9]:
import pandas as pd
import numpy as np
# Create pd_temp
pd_temp = pd.DataFrame(np.random.random(10))

# Create spark_temp from pd_temp
spark_temp = my_spark.createDataFrame(pd_temp)

# Examine the tables in the catalog
print(my_spark.catalog.listTables())

# Add spark_temp to the catalog
spark_temp.createOrReplaceTempView('temp')

# Examine the tables in the catalog again
print(my_spark.catalog.listTables())

[]
[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [11]:

# Don't change this file path
file_path = "covid_19_data.csv"

# Read in the corona data
corona = my_spark.read.csv(file_path,header=True)

# Show the data
print(corona.show())

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
|  1|     01/22/2020|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  2|     01/22/2020|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|
|  3|     01/22/2020|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|
|  4|     01/22/2020|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  5|     01/22/2020|         Gansu|Mainland China|1/22/2020 17:00|      0.0|   0.0|      0.0|
|  6|     01/22/2020|     Guangdong|Mainland China|1/22/2020 17:00|     26.0|   0.0|      0.0|
|  7|     01/22/2020|       Guangxi|Mainland China|1/22/2020 17:00|      2.0|   0.0|      0.0|
|  8|     01/22/2020|       Guizhou|Mainland China

In [12]:
print(my_spark.catalog.listTables())

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [13]:
corona.createOrReplaceTempView('Corona')

In [14]:
print(my_spark.catalog.listTables())

[Table(name='corona', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [16]:
# Create the DataFrame flights
c = my_spark.table('corona')

# Show the head
c.show()

# Add duration_hrs
c = c.withColumn('total',c.Confirmed+c.Deaths+ c.Recovered)
print(c.show())

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
|  1|     01/22/2020|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  2|     01/22/2020|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|
|  3|     01/22/2020|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|
|  4|     01/22/2020|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  5|     01/22/2020|         Gansu|Mainland China|1/22/2020 17:00|      0.0|   0.0|      0.0|
|  6|     01/22/2020|     Guangdong|Mainland China|1/22/2020 17:00|     26.0|   0.0|      0.0|
|  7|     01/22/2020|       Guangxi|Mainland China|1/22/2020 17:00|      2.0|   0.0|      0.0|
|  8|     01/22/2020|       Guizhou|Mainland China

In [17]:
c.filter('total>0').show()

+---+---------------+--------------+--------------+---------------+---------+------+---------+-----+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|total|
+---+---------------+--------------+--------------+---------------+---------+------+---------+-----+
|  1|     01/22/2020|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|  1.0|
|  2|     01/22/2020|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0| 14.0|
|  3|     01/22/2020|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|  6.0|
|  4|     01/22/2020|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|  1.0|
|  6|     01/22/2020|     Guangdong|Mainland China|1/22/2020 17:00|     26.0|   0.0|      0.0| 26.0|
|  7|     01/22/2020|       Guangxi|Mainland China|1/22/2020 17:00|      2.0|   0.0|      0.0|  2.0|
|  8|     01/22/2020|       Guizhou|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0

In [18]:
c.filter(c.total>0).show()

+---+---------------+--------------+--------------+---------------+---------+------+---------+-----+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|total|
+---+---------------+--------------+--------------+---------------+---------+------+---------+-----+
|  1|     01/22/2020|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|  1.0|
|  2|     01/22/2020|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0| 14.0|
|  3|     01/22/2020|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|  6.0|
|  4|     01/22/2020|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|  1.0|
|  6|     01/22/2020|     Guangdong|Mainland China|1/22/2020 17:00|     26.0|   0.0|      0.0| 26.0|
|  7|     01/22/2020|       Guangxi|Mainland China|1/22/2020 17:00|      2.0|   0.0|      0.0|  2.0|
|  8|     01/22/2020|       Guizhou|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0

In [30]:
filter1 = c['Country/Region'] == 'India'
filter2 = c.Deaths >0
x = c.select('Province/State','Deaths')
x.filter(filter1).filter(filter2).show()

+--------------+------+
|Province/State|Deaths|
+--------------+------+
|          null|   1.0|
|          null|   1.0|
|          null|   2.0|
|          null|   2.0|
|          null|   2.0|
|          null|   2.0|
|          null|   3.0|
|          null|   3.0|
|          null|   4.0|
|          null|   5.0|
|          null|   4.0|
|          null|   7.0|
|          null|  10.0|
|          null|  10.0|
|          null|  12.0|
|          null|  20.0|
|          null|  20.0|
|          null|  24.0|
|          null|  27.0|
|          null|  32.0|
+--------------+------+
only showing top 20 rows



In [34]:
c.filter(c['Country/Region']=='India').show()

+----+---------------+--------------+--------------+-------------------+---------+------+---------+-----+
| SNo|ObservationDate|Province/State|Country/Region|        Last Update|Confirmed|Deaths|Recovered|total|
+----+---------------+--------------+--------------+-------------------+---------+------+---------+-----+
| 431|     01/30/2020|          null|         India|      1/30/20 16:00|      1.0|   0.0|      0.0|  1.0|
| 492|     01/31/2020|          null|         India|    1/31/2020 23:59|      1.0|   0.0|      0.0|  1.0|
| 548|     02/01/2020|          null|         India|     1/31/2020 8:15|      1.0|   0.0|      0.0|  1.0|
| 608|     02/02/2020|          null|         India|2020-02-02T06:03:08|      2.0|   0.0|      0.0|  2.0|
| 673|     02/03/2020|          null|         India|2020-02-03T21:43:02|      3.0|   0.0|      0.0|  3.0|
| 741|     02/04/2020|          null|         India|2020-02-03T21:43:02|      3.0|   0.0|      0.0|  3.0|
| 811|     02/05/2020|          null|         

In [35]:
death_ratio_per_total_case = (c.Deaths/c.total).alias('death_ratio_per_total_case')
ratio1 = c.select('Country/Region',death_ratio_per_total_case)

In [36]:
ratio1.show()

+--------------+--------------------------+
|Country/Region|death_ratio_per_total_case|
+--------------+--------------------------+
|Mainland China|                       0.0|
|Mainland China|                       0.0|
|Mainland China|                       0.0|
|Mainland China|                       0.0|
|Mainland China|                      null|
|Mainland China|                       0.0|
|Mainland China|                       0.0|
|Mainland China|                       0.0|
|Mainland China|                       0.0|
|Mainland China|                       0.0|
|Mainland China|                      null|
|Mainland China|                       0.0|
|     Hong Kong|                      null|
|Mainland China|      0.034764826175869123|
|Mainland China|                       0.0|
|Mainland China|                      null|
|Mainland China|                       0.0|
|Mainland China|                       0.0|
|Mainland China|                      null|
|Mainland China|                

In [44]:
ratio2 = c.selectExpr('ObservationDate','Deaths/total as death_ratio')
ratio2.show()

+---------------+--------------------+
|ObservationDate|         death_ratio|
+---------------+--------------------+
|     01/22/2020|                 0.0|
|     01/22/2020|                 0.0|
|     01/22/2020|                 0.0|
|     01/22/2020|                 0.0|
|     01/22/2020|                null|
|     01/22/2020|                 0.0|
|     01/22/2020|                 0.0|
|     01/22/2020|                 0.0|
|     01/22/2020|                 0.0|
|     01/22/2020|                 0.0|
|     01/22/2020|                null|
|     01/22/2020|                 0.0|
|     01/22/2020|                null|
|     01/22/2020|0.034764826175869123|
|     01/22/2020|                 0.0|
|     01/22/2020|                null|
|     01/22/2020|                 0.0|
|     01/22/2020|                 0.0|
|     01/22/2020|                null|
|     01/22/2020|                 0.0|
+---------------+--------------------+
only showing top 20 rows



In [50]:
ratio2.groupBy().max('death_ratio').show()

+----------------+
|max(death_ratio)|
+----------------+
|             1.0|
+----------------+



In [54]:
c.groupBy('Country/Region').sum('total').show()

+--------------+----------+
|Country/Region|sum(total)|
+--------------+----------+
|          Chad|    1152.0|
|        Russia| 1377330.0|
|      Paraguay|    7525.0|
| North Ireland|       1.0|
|         Yemen|      50.0|
|       Senegal|   20785.0|
|        Sweden|  503124.0|
|    Cabo Verde|    1681.0|
|        Guyana|    2276.0|
|   Philippines|  209944.0|
|        Jersey|       6.0|
|       Eritrea|    1298.0|
|         Burma|    2940.0|
|      Djibouti|   22892.0|
|     Singapore|  245762.0|
|      Malaysia|  266550.0|
|          Fiji|     643.0|
|        Turkey| 2764353.0|
|        Malawi|     686.0|
|Western Sahara|     193.0|
+--------------+----------+
only showing top 20 rows



In [59]:
c.groupBy('ObservationDate').count().show()

+---------------+-----+
|ObservationDate|count|
+---------------+-----+
|     02/28/2020|  114|
|     03/22/2020|  298|
|     04/12/2020|  320|
|     02/11/2020|   73|
|     04/01/2020|  313|
|     04/08/2020|  320|
|     04/04/2020|  316|
|     04/27/2020|  321|
|     04/29/2020|  321|
|     04/14/2020|  320|
|     01/24/2020|   41|
|     02/25/2020|   94|
|     04/11/2020|  321|
|     03/16/2020|  272|
|     04/21/2020|  320|
|     02/15/2020|   75|
|     02/13/2020|   74|
|     03/10/2020|  206|
|     03/12/2020|  218|
|     02/18/2020|   75|
+---------------+-----+
only showing top 20 rows



In [60]:
c.groupBy('Country/Region').avg('total').show()

+--------------+------------------+
|Country/Region|        avg(total)|
+--------------+------------------+
|          Chad|26.181818181818183|
|        Russia|14970.978260869566|
|      Paraguay| 136.8181818181818|
| North Ireland|               1.0|
|         Yemen| 2.272727272727273|
|       Senegal|340.73770491803276|
|        Sweden| 5468.739130434783|
|    Cabo Verde|39.093023255813954|
|        Guyana|44.627450980392155|
|   Philippines| 2233.446808510638|
|        Jersey|              0.75|
|       Eritrea|30.904761904761905|
|         Burma| 81.66666666666667|
|      Djibouti| 508.7111111111111|
|     Singapore|           2457.62|
|      Malaysia|2692.4242424242425|
|          Fiji|14.613636363636363|
|        Turkey| 53160.63461538462|
|        Malawi|22.866666666666667|
|Western Sahara| 7.148148148148148|
+--------------+------------------+
only showing top 20 rows



In [62]:
import pyspark.sql.functions as F
by_country = c.groupBy('Country/Region')
by_country.avg('total').show()
by_country.agg(F.stddev('total')).show()

+--------------+------------------+
|Country/Region|        avg(total)|
+--------------+------------------+
|          Chad|26.181818181818183|
|        Russia|14970.978260869566|
|      Paraguay| 136.8181818181818|
| North Ireland|               1.0|
|         Yemen| 2.272727272727273|
|       Senegal|340.73770491803276|
|        Sweden| 5468.739130434783|
|    Cabo Verde|39.093023255813954|
|        Guyana|44.627450980392155|
|   Philippines| 2233.446808510638|
|        Jersey|              0.75|
|       Eritrea|30.904761904761905|
|         Burma| 81.66666666666667|
|      Djibouti| 508.7111111111111|
|     Singapore|           2457.62|
|      Malaysia|2692.4242424242425|
|          Fiji|14.613636363636363|
|        Turkey| 53160.63461538462|
|        Malawi|22.866666666666667|
|Western Sahara| 7.148148148148148|
+--------------+------------------+
only showing top 20 rows

+--------------+------------------+
|Country/Region|stddev_samp(total)|
+--------------+------------------+
| 

In [63]:
c.describe()

DataFrame[summary: string, SNo: string, ObservationDate: string, Province/State: string, Country/Region: string, Last Update: string, Confirmed: string, Deaths: string, Recovered: string, total: string]